In [1]:
import ee
import geemap
import os

In [2]:
ee.Authenticate()

True

In [3]:
# 1. Autenticación e inicialización de Earth Engine
ee.Initialize()  # Asumiendo que ya corriste ee.Authenticate() previamente

In [4]:
# ============================================================================
# CONFIGURACIÓN
# ============================================================================

# Bounding box en coordenadas geográficas (EPSG:4326)
# Formato: [xmin, ymin, xmax, ymax]
xmin = -68.803909
ymin = -11.033737
xmax = -68.725933
ymax = -10.979773

bbox = ee.Geometry.Rectangle([xmin, ymin, xmax, ymax], proj='EPSG:4326', geodesic=False)

# Rango de fechas
start_date = '2017-09-01'
end_date   = '2026-01-23'

# Parámetros de exportación
scale = 10  # metros

# CRS UTM de salida (AJUSTA ESTO A TU ZONA)
utm_epsg = 'EPSG:32719'  # por ejemplo UTM 18N

# Carpetas de salida
output_root = r"C:\SPA_SEI\SENTINEL1_DESCARGAS"
output_dir_vv = os.path.join(output_root, "VV")
output_dir_vh = os.path.join(output_root, "VH")

os.makedirs(output_dir_vv, exist_ok=True)
os.makedirs(output_dir_vh, exist_ok=True)

# Filtros Sentinel-1
collection_id = 'COPERNICUS/S1_GRD'
orbit_filter_pass = 'DESCENDING'  # 'ASCENDING', 'DESCENDING' o None
relative_orbit = 127     # número de órbita relativa o None

In [5]:
# ============================================================================
# FUNCIONES AUXILIARES
# ============================================================================

def prepare_collection():
    """Prepara la colección Sentinel-1 filtrada y ordenada."""
    col = (ee.ImageCollection(collection_id)
           .filterBounds(bbox)
           .filterDate(start_date, end_date)
           .filter(ee.Filter.eq('instrumentMode', 'IW'))
           .filter(ee.Filter.eq('productType', 'GRD'))
           .filter(ee.Filter.eq('platform_number', 'A'))
           )

    if orbit_filter_pass is not None:
        col = col.filter(ee.Filter.eq('orbitProperties_pass', orbit_filter_pass))

    # Asegurar VV y VH
    col = col.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
             .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))

    if relative_orbit is not None:
        col = col.filter(ee.Filter.eq('relativeOrbitNumber_start', relative_orbit))

    col = col.sort('system:time_start')
    return col

def export_image_local(image, pol, out_dir, date_str, orbit_char):
    """
    Exporta una banda (VV o VH) a un GeoTIFF local usando geemap.
    pol: 'VV' o 'VH'
    orbit_char: 'A' o 'D'
    """
    filename = os.path.join(out_dir, f"{date_str}_{pol}_{orbit_char}.tif")
    print(f"Descargando {filename} ...")

    geemap.ee_export_image(
        image,              # argumento posicional
        filename=filename,
        scale=scale,
        region=bbox,        # región en WGS84
        crs=utm_epsg,       # reproyección a UTM en la exportación
        file_per_band=False
    )

In [6]:
# ============================================================================
# PROCESO PRINCIPAL
# ============================================================================

def main():
    col = prepare_collection()
    size = col.size().getInfo()
    print(f"Imágenes encontradas con los filtros dados: {size}")

    if size == 0:
        print("No se encontraron imágenes para los filtros especificados.")
        return

    # Para prueba rápida, descomenta:
    # size = min(size, 3)

    img_list = col.toList(size)

    for i in range(size):
        img = ee.Image(img_list.get(i))

        # Fecha para el nombre del archivo
        date = ee.Date(img.get('system:time_start'))
        date_str = date.format('yyyyMMdd').getInfo()

        # Tipo de órbita (ASCENDING/DESCENDING) y letra A/D
        orbit_pass = ee.String(img.get('orbitProperties_pass')).getInfo()
        orbit_char = 'A' if orbit_pass.upper().startswith('ASC') else 'D'

        # Bandas VV y VH (ya en dB en COPERNICUS/S1_GRD)
        vv = img.select('VV')
        vh = img.select('VH')

        # Exportar con nombres YYYYMMDD_VV/VH_A/D
        export_image_local(vv, "VV", output_dir_vv, date_str, orbit_char)
        export_image_local(vh, "VH", output_dir_vh, date_str, orbit_char)

    print("Proceso terminado.")

# Ejecutar
if __name__ == "__main__":
    main()

Imágenes encontradas con los filtros dados: 252
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20170904_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20170904_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20170904_VH_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20170904_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20170916_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20170916_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20170916_VH_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20170916_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20170928_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20170928_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20170928_VH_D.tif ...
Generating URL

Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20180126_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20180207_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20180207_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20180207_VH_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20180207_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20180219_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20180219_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20180219_VH_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20180219_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20180303_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20180303_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20180303_VH_D

Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20180619_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20180701_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20180701_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20180701_VH_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20180701_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20180713_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20180713_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20180713_VH_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20180713_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20180725_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20180725_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20180725_VH_D

Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20181110_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20181122_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20181122_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20181122_VH_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20181122_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20181204_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20181204_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20181204_VH_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20181204_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20181216_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20181216_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20181216_VH_D

Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20190403_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20190415_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20190415_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20190415_VH_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20190415_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20190427_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20190427_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20190427_VH_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20190427_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20190509_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20190509_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20190509_VH_D

...

Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20231214_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20231226_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20231226_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20231226_VH_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20231226_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20240107_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20240107_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20240107_VH_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20240107_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20240119_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20240119_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20240119_VH_D

Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20240506_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20240518_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20240518_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20240518_VH_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20240518_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20240530_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20240530_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20240530_VH_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20240530_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20240611_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20240611_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20240611_VH_D

Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20240927_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20241009_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20241009_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20241009_VH_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20241009_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20241021_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20241021_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20241021_VH_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20241021_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20241102_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20241102_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20241102_VH_D

Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20250218_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20250314_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20250314_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20250314_VH_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20250314_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20250326_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20250326_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20250326_VH_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20250326_VH_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20250407_VV_D.tif ...
Generating URL ...
Please wait ...
Data downloaded to C:\SPA_SEI\SENTINEL1_DESCARGAS\VV\20250407_VV_D.tif
Descargando C:\SPA_SEI\SENTINEL1_DESCARGAS\VH\20250407_VH_D

EEException: Project ee-sebastianpalominoangel is not registered to use Earth Engine. Visit https://console.cloud.google.com/earth-engine/configuration?project=ee-sebastianpalominoangel to register your project. See https://developers.google.com/earth-engine/guides/access for more details. If you have previously registered your project for noncommercial use and lost access, you will need to re-verify your eligibility. See the https://developers.google.com/earth-engine/guides/access#configuring_noncommercial_access for more details.